## mounting google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## clear cuda

In [ ]:
import torch
torch.cuda.empty_cache()
import gc
gc.collect()
torch.cuda.empty_cache()


## Install

In [ ]:
!pip install jiwer
!pip install openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 54.4 MB/s eta 0:00:00


## imports

In [ ]:
import pandas as pd
from transformers import MT5Tokenizer, MT5ForConditionalGeneration
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset,DataLoader
from torch.optim import Adam
from torch.nn.utils.rnn import pad_sequence
import torch.nn as nn
from torch.nn.functional import pad as pad_tokens

## determining environment


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


## Define paths

In [ ]:
FILE_PATH='/content/drive/MyDrive/ASR_Hindi_Training_Data.xlsx'

## Reading Dataset

In [ ]:
df = pd.read_excel(FILE_PATH)
print(df.columns)
df

Index(['Output from ASR', 'Correct Sentence'], dtype='object')


,Output from ASR,Correct Sentence
0,जयपुर खुदकुशी करने वाले रेवपीरिता परी पुलिस ने...,जयपुर खुदकुशी करने वाली रेप पीड़िता पर ही पुलि...
1,राजस्थान चुनाव कीटड़ी में क्या वापस कर पाएंगी ...,राजस्थान चुनाव केकड़ी में क्या वापसी कर पाएगी ...
2,विोविदेशों में जमा तला कालाधन ो की फाइलिंग से ...,विदेशों में जमा कालाधन को ई फाइलिंग से करें घो...
3,फिल् डायरेक्शन के लिए धर्म जरूरी नहीं अक्षा है,फिल्म डायरेक्शन के लिए धर्म जरूरी नहीं अक्षय
4,आकर्षक और सुदौल भार की सलक,आकर्षक और सुडौल उभार की सनक
...,...,...
2867,कमा रे कोक सा के किसमे है,तुम्हारे बहुत सारे दुशमन हैं
2868,छेड़छाण का विरोध कर रहे छात्रों को पुलिस ने पीट...,छेड़छाड़ का विरोध कर रहे छात्रों को पुलिस ने प...
2869,विराटकोली के लिए पेह रिनमाउका,<unk>pinion विराट कोहली के लिए बेहतरीन मौका
2870,बॉक्स ऑफिस का नया हीरोबना होर लेलामजनू पलटन पीछे,<unk>o<unk> office का नया हीरो बना हॉरर लैला म...


## temporary splitting dataset for try out with 50 rows

In [ ]:
# df = df[0:50]
df

,Output from ASR,Correct Sentence
0,जयपुर खुदकुशी करने वाले रेवपीरिता परी पुलिस ने...,जयपुर खुदकुशी करने वाली रेप पीड़िता पर ही पुलि...
1,राजस्थान चुनाव कीटड़ी में क्या वापस कर पाएंगी ...,राजस्थान चुनाव केकड़ी में क्या वापसी कर पाएगी ...
2,विोविदेशों में जमा तला कालाधन ो की फाइलिंग से ...,विदेशों में जमा कालाधन को ई फाइलिंग से करें घो...
3,फिल् डायरेक्शन के लिए धर्म जरूरी नहीं अक्षा है,फिल्म डायरेक्शन के लिए धर्म जरूरी नहीं अक्षय
4,आकर्षक और सुदौल भार की सलक,आकर्षक और सुडौल उभार की सनक
...,...,...
2867,कमा रे कोक सा के किसमे है,तुम्हारे बहुत सारे दुशमन हैं
2868,छेड़छाण का विरोध कर रहे छात्रों को पुलिस ने पीट...,छेड़छाड़ का विरोध कर रहे छात्रों को पुलिस ने प...
2869,विराटकोली के लिए पेह रिनमाउका,<unk>pinion विराट कोहली के लिए बेहतरीन मौका
2870,बॉक्स ऑफिस का नया हीरोबना होर लेलामजनू पलटन पीछे,<unk>o<unk> office का नया हीरो बना हॉरर लैला म...


## Seperating input output

In [ ]:
X,y = df['Output from ASR'],df['Correct Sentence']
X=X.tolist()
y=y.tolist()
print("X",X)
print("y",y)

X ['जयपुर खुदकुशी करने वाले रेवपीरिता परी पुलिस ने मड दिया सारा दोश ', 'राजस्थान चुनाव कीटड़ी में क्या वापस कर पाएंगी कांग्रेस ', 'विोविदेशों में जमा तला कालाधन ो की फाइलिंग से करे घोषित ', 'फिल् डायरेक्शन के लिए धर्म जरूरी नहीं अक्षा है ', 'आकर्षक और सुदौल भार की सलक ', 'अमीरखान की पीके के ऑडियू टीजर का प्रमोषण पीटर परे ', 'अरट ऑफ लीजिंग फाउंडेशन को मिली धंम की ', 'सफाई गिरी स्वछ कुमार ', 'फेसबू का इन्टरनेट देने वालासोल ड्रोग परीक्षण के लिए तैयार ', 'भारा आटोमि्रीसर सेंटर मे विकैनसी ', 'वायानाड कांग्रेस की इस बेहद सुरक्षित सीट पर दिग्घज नेताओं की नजेय ', 'यंग अमीक्षा का दिल जू न कर सका ', 'त्रायं्ब केश्वर मंदिर में प्रवेश के बाद हिरासत लेली गई तृप्ति देसाई ', 'चुनाव आयोग ने जारी किया क्या करें और क्या लह करें ', 'भारच के युदध का खतरा नहीं किलानि ', 'जानमार अतंकियों को मारने वाले ये है कमाल डू ', 'तस्करी के लिए लिए जा रहे थे गाय पुलिस पर की फायडी ', 'अमेरिका के साथ अच्छे संबंध चाहते हैं जिलानी ', 'सरा सुरक्षा परिषद में जल सुधार की उम्मीद नहीं अमेरिका ', 'बिहार केदी को पेशी के लिए लिए ज

### Preprocessing data
* After above tryout, found that some white spacing and all issues, so we need to trim the data first


In [ ]:
# trim each element of X and y
X = [sentence.strip() for sentence in X]
y = [sentence.strip() for sentence in y]
print("X",X)
print("y",y)

X ['जयपुर खुदकुशी करने वाले रेवपीरिता परी पुलिस ने मड दिया सारा दोश', 'राजस्थान चुनाव कीटड़ी में क्या वापस कर पाएंगी कांग्रेस', 'विोविदेशों में जमा तला कालाधन ो की फाइलिंग से करे घोषित', 'फिल् डायरेक्शन के लिए धर्म जरूरी नहीं अक्षा है', 'आकर्षक और सुदौल भार की सलक', 'अमीरखान की पीके के ऑडियू टीजर का प्रमोषण पीटर परे', 'अरट ऑफ लीजिंग फाउंडेशन को मिली धंम की', 'सफाई गिरी स्वछ कुमार', 'फेसबू का इन्टरनेट देने वालासोल ड्रोग परीक्षण के लिए तैयार', 'भारा आटोमि्रीसर सेंटर मे विकैनसी', 'वायानाड कांग्रेस की इस बेहद सुरक्षित सीट पर दिग्घज नेताओं की नजेय', 'यंग अमीक्षा का दिल जू न कर सका', 'त्रायं्ब केश्वर मंदिर में प्रवेश के बाद हिरासत लेली गई तृप्ति देसाई', 'चुनाव आयोग ने जारी किया क्या करें और क्या लह करें', 'भारच के युदध का खतरा नहीं किलानि', 'जानमार अतंकियों को मारने वाले ये है कमाल डू', 'तस्करी के लिए लिए जा रहे थे गाय पुलिस पर की फायडी', 'अमेरिका के साथ अच्छे संबंध चाहते हैं जिलानी', 'सरा सुरक्षा परिषद में जल सुधार की उम्मीद नहीं अमेरिका', 'बिहार केदी को पेशी के लिए लिए जा रहे पुलिस करमी की

### Train & Test split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## mbart

In [ ]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50")
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50")

# Set correct language codes
tokenizer.src_lang = "hi_IN"
tokenizer.tgt_lang = "hi_IN"


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/531 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

### tokenizing some sentences to analyze what is the ouptut and how tokenized input output looks like

In [ ]:
sentence = "मैं कल दोपहर 2 बजे मीटिंग में हूँ।"
print("Sentence : ",sentence)
tokenized_sentence = tokenizer(sentence)
print("Tokenized Sentence : ",tokenized_sentence)

Sentence :  मैं कल दोपहर 2 बजे मीटिंग में हूँ।
Tokenized Sentence :  {'input_ids': [250010, 10399, 32587, 233452, 116, 34554, 8415, 72123, 421, 28035, 125, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


### Tokenize Data

In [ ]:
def calculate_max_token_length(X, y, tokenizer):
    # Tokenize all input and target sentences without padding or truncation
    input_lengths = [len(tokenizer.encode(sentence)) for sentence in X]
    target_lengths = [len(tokenizer.encode(sentence)) for sentence in y]

    # Find the maximum length
    max_input_length = max(input_lengths)
    max_target_length = max(target_lengths)

    # Use the larger of the two as the max token length
    max_token_length = max(max_input_length, max_target_length)

    return max_token_length

# Calculate max token length
max_token_length = calculate_max_token_length(X_train + X_test, y_train + y_test, tokenizer)
print(f"Max token length: {max_token_length}")

Max token length: 40


In [ ]:
def tokenize_data(X, y, tokenizer, max_length=max_token_length):
    X = [f"{sentence}" for sentence in X]
    inputs = tokenizer(X, padding="max_length", max_length=max_length, truncation=True, return_tensors="pt")
    targets = tokenizer(y, padding="max_length", max_length=max_length, truncation=True, return_tensors="pt")
    return inputs['input_ids'], targets['input_ids']

# Tokenize training and testing data
X_train_tokenized, y_train_tokenized = tokenize_data(X_train, y_train, tokenizer)
X_test_tokenized, y_test_tokenized = tokenize_data(X_test, y_test, tokenizer)

In [ ]:
print(X_train[0])
print(y_train[0])
print(X_train_tokenized[0])
print(y_train_tokenized[0])
print(X_test_tokenized[0])
print(y_test_tokenized[0])

और क्या है
और क्या है
tensor([250010,    871,   6004,    460,      2,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1])
tensor([250010,    871,   6004,    460,      2,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1])
tensor([250010, 236827,    421,  10195,   7505,   9729,   5006,   8906,      6,
          3045, 120763,    659,   5093,  66407,      2,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1, 

## Loading model

In [ ]:
import os


In [ ]:
from transformers import MBartForConditionalGeneration
import torch

# Load the model and tokenizer
# model = MT5ForConditionalGeneration.from_pretrained("google/mt5-small")
model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50")

# Set device
device = "cuda" if torch.cuda.is_available() else "cpu"
# os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
model = model.to(device)

print("Model and tokenizer loaded successfully!")

Model and tokenizer loaded successfully!


In [ ]:
def correct_asr(text, model, tokenizer, device):
    input_text = f"{text}"  #prefix for T5 to understand the task
    inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)
    print(inputs)

    # Generate corrected text
    with torch.no_grad():
        generated_ids = model.generate(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'], max_length=512)

    # Decode the generated text
    corrected_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
    return corrected_text.lower()

# Example inputs
input_texts = [
     "मैं कल दोपहर 2 बजे मीटिंग में हूँ।",
      "वह बाद में स्टोर जाएगी।"
]

# Perform inference on example inputs
for text in input_texts:
    corrected_text = correct_asr(text, model, tokenizer, device)
    print(f"Input: {text}")
    print(f"Corrected: {corrected_text}")
    print("-" * 50)

{'input_ids': tensor([[250010,  10399,  32587, 233452,    116,  34554,   8415,  72123,    421,
          28035,    125,      2]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}
Input: मैं कल दोपहर 2 बजे मीटिंग में हूँ।
Corrected: मैं कल दोपहर 2 बजे मीटिंग में हूँ
--------------------------------------------------
{'input_ids': tensor([[250010,  11692,   6435,    421, 191745, 102780,    125,      2]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}
Input: वह बाद में स्टोर जाएगी।
Corrected: वह बाद में स्टोर जाएगी।
--------------------------------------------------


In [ ]:
print(tokenizer.special_tokens_map)
print(tokenizer.convert_ids_to_tokens([9422, 2917, 71757, 6904, 3863, 20310, 356, 1822, 10561, 29829, 59000, 844, 2768, 48108, 378, 1]))  # Check what these tokens actually mean


{'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>', 'additional_special_tokens': ['ar_AR', 'cs_CZ', 'de_DE', 'en_XX', 'es_XX', 'et_EE', 'fi_FI', 'fr_XX', 'gu_IN', 'hi_IN', 'it_IT', 'ja_XX', 'kk_KZ', 'ko_KR', 'lt_LT', 'lv_LV', 'my_MM', 'ne_NP', 'nl_XX', 'ro_RO', 'ru_RU', 'si_LK', 'tr_TR', 'vi_VN', 'zh_CN', 'af_ZA', 'az_AZ', 'bn_IN', 'fa_IR', 'he_IL', 'hr_HR', 'id_ID', 'ka_GE', 'km_KH', 'mk_MK', 'ml_IN', 'mn_MN', 'mr_IN', 'pl_PL', 'ps_AF', 'pt_XX', 'sv_SE', 'sw_KE', 'ta_IN', 'te_IN', 'th_TH', 'tl_XX', 'uk_UA', 'ur_PK', 'xh_ZA', 'gl_ES', 'sl_SI']}
['▁სი', '▁б', '▁bír', 'ಣ', '▁ch', 'ದ್ದ', 'को', '▁dla', '▁gjort', '▁hedef', 'APA', '等', '▁شما', 'ιερ', '▁[', '<pad>']


## Dataset

In [ ]:
class ASRDataset(Dataset):
    def __init__(self, inputs, targets):
        self.inputs = inputs
        self.targets = targets

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return (self.inputs[idx]), (self.targets[idx])

train_dataset = ASRDataset(X_train_tokenized,y_train_tokenized)
test_dataset = ASRDataset(X_test_tokenized,y_test_tokenized)

train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=True)

print("ith item of dataset : ",train_dataset[3])
print("length of dataset : ",len(train_dataset))


ith item of dataset :  (tensor([250010,  55471,  46547,  11635, 159953,  28998,   6960,  91462,  27497,
          2138,   1142,   1780,  10195,  14983,   3889,  20914,   1026, 107360,
             2,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1]), tensor([250010,  55471,  46547,  11635, 159953,  28998, 127096,  16338,    998,
           871,  35993,  16338,   2138,  85134,   1142,   1780,  10195,  14983,
          3889,  20914,   1026, 107360,      2,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1]))
length of dataset :  2297


In [ ]:
num_epochs = 3
learning_rate = 5e-5

# Optimizer
optimizer = Adam(model.parameters(), lr=learning_rate)

# Loss function
# criterion = nn.MSELoss()
criterion = nn.CrossEntropyLoss()
# import torch.nn.functional as F
# criterion = F.mse_loss

optimizer
criterion

CrossEntropyLoss()

### code to get accuracy on test data

In [ ]:
!pip install jiwer
import numpy as np
from jiwer import wer, cer

def train(model, dataloader, optimizer, device, tokenizer):
    model.train()
    total_loss = 0
    total_tokens = 0
    matched_tokens = 0
    total_wer = 0
    total_cer = 0
    total_sentences = 0
    incorrect_sentences = 0

    for batch_idx, batch in enumerate(dataloader):
        optimizer.zero_grad()

        input_ids = batch[0].to(device)
        attention_mask = (input_ids != tokenizer.pad_token_id).int().to(device)
        labels = batch[1].to(device)

        # Forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        # Generate model's output
        with torch.no_grad():
            generated_ids = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=max_token_length)

        # Decode input, target, and generated output
        input_text = tokenizer.decode(input_ids[0], skip_special_tokens=True)
        target_text = tokenizer.decode(labels[0], skip_special_tokens=True)
        generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

        # Convert target and generated text to lowercase
        target_text = target_text.lower()
        generated_text = generated_text.lower()

        # Print tokenized input and output for debugging
        # if batch_idx % 10 == 0:
            # print(f"Tokenized Input IDs: {input_ids[0]}")
            # print(f"Tokenized Target IDs: {labels[0]}")
            # print(f"Generated Token IDs: {generated_ids[0]}")

        # Calculate token-level accuracy
        predicted_tokens = generated_ids[0]
        expected_tokens = labels[0]

        # Find the minimum length of the two tensors
        min_length = min(len(predicted_tokens), len(expected_tokens))

        # Truncate both tensors to the minimum length
        predicted_tokens = predicted_tokens[:min_length]
        expected_tokens = expected_tokens[:min_length]

        curr_total_tokens = len(expected_tokens)
        curr_matched_tokens = (predicted_tokens == expected_tokens).sum().item()

        total_tokens += curr_total_tokens
        matched_tokens += curr_matched_tokens

        # Calculate WER and CER
        curr_wer = wer(target_text, generated_text)
        curr_cer = cer(target_text, generated_text)

        total_wer += curr_wer
        total_cer += curr_cer

        # Calculate SER
        if generated_text != target_text:
            incorrect_sentences += 1
        total_sentences += 1

        # Print input, target, and generated output every few batches (e.g., every 10 batches)
        if batch_idx % 10 == 0:
            print(f"Batch {batch_idx}:")
            print(f"Input: {input_text}")
            print(f"Target: {target_text}")
            print(f"Generated: {generated_text}")
            print(f"WER: {curr_wer:.4f}, CER: {curr_cer:.4f}")
            print("-" * 50)

    # Calculate overall metrics
    accuracy = (matched_tokens / total_tokens) * 100.0
    avg_wer = (total_wer / total_sentences) * 100.0
    avg_cer = (total_cer / total_sentences) * 100.0
    ser = (incorrect_sentences / total_sentences) * 100.0

    print(f"Training Metrics:")
    print(f"Accuracy: {accuracy:.2f}%")
    print(f"Average WER: {avg_wer:.2f}%")
    print(f"Average CER: {avg_cer:.2f}%")
    print(f"SER: {ser:.2f}%")
    print("=" * 50)

    return total_loss / len(dataloader)

# Training for a few epochs
for epoch in range(3):
    train_loss = train(model, train_loader, optimizer, device, tokenizer)
    print(f"Epoch {epoch + 1}, Loss: {train_loss}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 38.9 MB/s eta 0:00:00
Batch 0:
Input: सुंखश है] n
Target: हम खुश हैं
Generated: सुंखश हैश है
WER: 1.0000, CER: 0.8000
--------------------------------------------------
Batch 10:
Input: फ़ेसबुक ट्वीटर और वॉच्साप पर अफ़वा फ़ैलाने वालों की खैर नईं
Target: फेसबुक ट्विटर और व्हॉट्सऐप पर अफवाह फैलाने वालों की खैर नहीं
Generated: फ़ेसबुक ट्वीटर और वॉच्साप पर अफ़वा फ़ैलाने वालों की खैर नईं
WER: 0.5455, CER: 0.1833
--------------------------------------------------
Batch 20:
Input: कडने के लिए हमेशा कुछ न कुछ तो होता ही है
Target: करने के लिए हमेशा कुछ न कुछ तो होता ही है
Generated: कडने के लिए हमेशा कुछ न कुछ तो होता है
WER: 0.1818, CER: 0.0976
--------------------------------------------------
Batch 30:
Input: थोडी सी चाई बनाऊ
Target: थोड़ी सी चाय बनाऊं
Generated: थोडी सी चाई बनाऊ
WER: 0.7500, CER: 0.1667
--------------------------------------------------
Batch 40:
Input: इन सवालों का जवाब देना आसान है
Target: इन सवालों का जवाब देना आसान

In [ ]:
for epoch in range(1):
    train_loss = train(model, train_loader, optimizer, device, tokenizer)
    print(f"Epoch {epoch + 1}, Loss: {train_loss}")

Batch 0:
Input: अमेरीका में फटा एप्पल का एयरपॉर्ट जांच शुरू
Target: अमेरिका में फटा apple का air od जांच शुरू
Generated: अमेरिका में फटा apple का air
WER: 0.3333, CER: 0.3171
--------------------------------------------------
Batch 10:
Input: बसटान की बात सुलह
Target: बस टॉम की बात सुनो
Generated: बस टॉम की बात सुनो
WER: 0.0000, CER: 0.0000
--------------------------------------------------
Batch 20:
Input: पाक को शईद के खिलाफ कार्रवाई करने की जरूरत हू
Target: पाक को सईद के खिलाफ कार्रवाई करने की जरूरत ईयू
Generated: पाक को ईद के खिलाफ कार्रवाई करने की जरूरत
WER: 0.2000, CER: 0.1087
--------------------------------------------------
Batch 30:
Input: बोलो तुम ऐसा क्यों करना चाहते हों
Target: बोलो तुम ऐसा क्यों करना चाहते हो
Generated: बोलो तुम ऐसा क्यों करना चाहते हो
WER: 0.0000, CER: 0.0000
--------------------------------------------------
Batch 40:
Input: तुम भी जूट बोल रही हो है
Target: तुम अभी झूठ बोल रही हो है ना
Generated: तुम अभी झूठ बोल रही हो है
WER: 0.1250, CER: 0.1071
------

## Save Model

In [ ]:
import os
import torch

# Define the directory to save the model
save_directory = "/content/drive/MyDrive/ASR_T5_Model"

# Create the directory if it doesn't exist
os.makedirs(save_directory, exist_ok=True)

model_path = os.path.join(save_directory, "mbart_asr_model_hindi.pth")
torch.save(model.state_dict(), model_path)

# Save the tokenizer
tokenizer_path = os.path.join(save_directory, "mbart_asr_model_hindi")
tokenizer.save_pretrained(tokenizer_path)

print(f"Model saved  {model_path}")
print(f"Tokenizer saved at {tokenizer_path}")


Model saved  /content/drive/MyDrive/ASR_T5_Model/mbart_asr_model_hindi.pth
Tokenizer saved at /content/drive/MyDrive/ASR_T5_Model/mbart_asr_model_hindi


## Trained on Custom Dataset:

In [ ]:
import pandas as pd
from transformers import T5Tokenizer, T5ForConditionalGeneration
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
import torch.nn as nn
from jiwer import wer, cer

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Define paths
MODEL_PATH = "/content/drive/MyDrive/ASR_T5_Model/mbart_asr_model_hindi.pth"
TOKENIZER_PATH = "/content/drive/MyDrive/ASR_T5_Model/mbart_asr_model_hindi"
NEW_DATASET_PATH = "/content/drive/MyDrive/datasets/asr_correction_dataset_2_cleaned.csv"

# Move model to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load the saved model and tokenizer
tokenizer = T5Tokenizer.from_pretrained(TOKENIZER_PATH)

model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base")  # Use the base model or your custom model
model.load_state_dict(torch.load(MODEL_PATH, map_location=device))

model.to(device)
model.eval()

# Load the new dataset
df = pd.read_csv(NEW_DATASET_PATH)
print(df.head())

# Preprocess the dataset
X = df['predicted_transcript'].tolist()
y = df['actual_sentence'].tolist()

# Trim and lowercase the data
X = [sentence.lower().strip() for sentence in X]
y = [sentence.lower().strip() for sentence in y]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Tokenize the data
def calculate_max_token_length(X, y, tokenizer):
    input_lengths = [len(tokenizer.encode(sentence)) for sentence in X]
    target_lengths = [len(tokenizer.encode(sentence)) for sentence in y]
    max_token_length = max(max(input_lengths), max(target_lengths))
    return max_token_length

max_token_length = calculate_max_token_length(X_train + X_test, y_train + y_test, tokenizer)
print(f"Max token length: {max_token_length}")

def tokenize_data(X, y, tokenizer, max_length=max_token_length):
    inputs = tokenizer(X, padding="max_length", max_length=max_length, truncation=True, return_tensors="pt")
    targets = tokenizer(y, padding="max_length", max_length=max_length, truncation=True, return_tensors="pt")
    return inputs['input_ids'], targets['input_ids']

X_train_tokenized, y_train_tokenized = tokenize_data(X_train, y_train, tokenizer)
X_test_tokenized, y_test_tokenized = tokenize_data(X_test, y_test, tokenizer)

# Dataset class
class ASRDataset(Dataset):
    def __init__(self, inputs, targets):
        self.inputs = inputs
        self.targets = targets

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return self.inputs[idx], self.targets[idx]

train_dataset = ASRDataset(X_train_tokenized, y_train_tokenized)
test_dataset = ASRDataset(X_test_tokenized, y_test_tokenized)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

def train(model, dataloader, optimizer, device, tokenizer):
    model.train()
    total_loss = 0
    total_tokens = 0
    matched_tokens = 0
    total_wer = 0
    total_cer = 0
    total_sentences = 0
    incorrect_sentences = 0

    for batch_idx, batch in enumerate(dataloader):
        optimizer.zero_grad()

        input_ids = batch[0].to(device)
        attention_mask = (input_ids != tokenizer.pad_token_id).int().to(device)
        labels = batch[1].to(device)

        # Forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        # Generate model's output
        with torch.no_grad():
            generated_ids = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=max_token_length)

        # Decode input, target, and generated output
        input_text = tokenizer.decode(input_ids[0], skip_special_tokens=True)
        target_text = tokenizer.decode(labels[0], skip_special_tokens=True)
        generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

        # Convert target and generated text to lowercase
        target_text = target_text.lower()
        generated_text = generated_text.lower()

        # Print tokenized input and output for debugging
        if batch_idx % 10 == 0:
            print(f"Tokenized Input IDs: {input_ids[0]}")
            print(f"Tokenized Target IDs: {labels[0]}")
            print(f"Generated Token IDs: {generated_ids[0]}")

        # Calculate token-level accuracy
        predicted_tokens = generated_ids[0]
        expected_tokens = labels[0]

        # Find the minimum length of the two tensors
        min_length = min(len(predicted_tokens), len(expected_tokens))

        # Truncate both tensors to the minimum length
        predicted_tokens = predicted_tokens[:min_length]
        expected_tokens = expected_tokens[:min_length]

        curr_total_tokens = len(expected_tokens)
        curr_matched_tokens = (predicted_tokens == expected_tokens).sum().item()

        total_tokens += curr_total_tokens
        matched_tokens += curr_matched_tokens

        # Calculate WER and CER
        curr_wer = wer(target_text, generated_text)
        curr_cer = cer(target_text, generated_text)

        total_wer += curr_wer
        total_cer += curr_cer

        # Calculate SER
        if generated_text != target_text:
            incorrect_sentences += 1
        total_sentences += 1

        # Print input, target, and generated output every few batches (e.g., every 10 batches)
        if batch_idx % 10 == 0:
            print(f"Batch {batch_idx}:")
            print(f"Input: {input_text}")
            print(f"Target: {target_text}")
            print(f"Generated: {generated_text}")
            print(f"WER: {curr_wer:.4f}, CER: {curr_cer:.4f}")
            print("-" * 50)

    # Calculate overall metrics
    accuracy = (matched_tokens / total_tokens) * 100.0
    avg_wer = (total_wer / total_sentences) * 100.0
    avg_cer = (total_cer / total_sentences) * 100.0
    ser = (incorrect_sentences / total_sentences) * 100.0

    print(f"Training Metrics:")
    print(f"Accuracy: {accuracy:.2f}%")
    print(f"Average WER: {avg_wer:.2f}%")
    print(f"Average CER: {avg_cer:.2f}%")
    print(f"SER: {ser:.2f}%")
    print("=" * 50)

    return total_loss / len(dataloader)

# Training for a few epochs
for epoch in range(3):
    train_loss = train(model, train_loader, optimizer, device, tokenizer)
    print(f"Epoch {epoch + 1}, Loss: {train_loss}")

# Save the fine-tuned model
fine_tuned_model_path = "/content/drive/MyDrive/ASR_T5_Model/flan_t5_asr_model_fine_tuned.pth"
torch.save(model.state_dict(), fine_tuned_model_path)
print(f"Fine-tuned model saved to {fine_tuned_model_path}")

# Evaluate Code

In [ ]:
input_file_path = "/content/drive/MyDrive/ASR_EVAL_DATA/hindi_eval.txt"

EVAL_FILE_PATH = input_file_path
f = open(EVAL_FILE_PATH, "r")
eval_content = f.read()
X_eval = eval_content.split('\n')
print(len(X_eval))
print(max([len(x) for x in X_eval]))


1001
100


In [ ]:
def split_sentence(sentence, max_words):
  splitted_sentences = []
  for i in range(0, len(sentence.split()), max_words):
    curr_sentence = ' '.join(sentence.split()[i:i+max_words])
    splitted_sentences.append(curr_sentence)
  return splitted_sentences


In [ ]:
X_eval_splitted = []
for sentence in X_eval:
  X_eval_splitted.append(split_sentence(sentence,10))
X_eval_splitted

[['बच्चरे बच्चरियों के नाभी का सरना क्या होता है?'],
 ['गाय के अफ़ारा लोग से कैसे बचाओ करना चाहिए'],
 ['कर्थ नकट की नुकसान की प्रवरिती एवं निशान बताईए'],
 ['गेहु की और सिंचिद्धसा प्रजाती के बारे में बताएं'],
 ['जौ के जूज़ा या पत्ती धबारो किसे कहते हैं?'],
 ['पिला रश्ट हर दा का जैविक नियंधन कैसे करेंगे?'],
 ['मुंग की पंत मुंग एक किसन की जानकारी दीजे'],
 ['जूट की JRC 698 किसमे के बारे में बताईए'],
 ['लूज उसमट रोग से नियंत्रन की रसाइनिक विधी बताएं।'],
 ['हर हर दाल का भीजो पशार कैसे करनी चाहिए'],
 ['जौर के पौधों को बिरली करन की जनकारी दीजिये'],
 ['गेहु की आसिंचित दसा प्रजाती के बारे में बताईए।'],
 ['टमाटर के पवदों पर मिट्टी चड़ाना क्यू आवशक है?'],
 ['जड़ विलगन रोक की रोक थाम कैसे करनी चाहिए?'],
 ['सिवेरियम रोग के पर पोशी फसरों के नाम बताएँ'],
 ['अमरोद की पौधा रोपन विधी के बारे में बताईए'],
 ['संत्रे की खेती के लिए पौध रोपन विधी बताईए'],
 ['सरिफा की पौदों की देख रेक किसी करने चाहिए'],
 ['जो में नाइट्रोजन, फास्फोरस, प्रेस्पोटास कप तथा कैसे डाले?'],
 ['मुख की खेते के लिए बुआय की विधी बताईए'],

In [ ]:
def infer(model, sentence):
  tokenized_sentence = tokenizer(sentence, return_tensors="pt")
  input_ids=(tokenized_sentence['input_ids']).to(device)
  attention_mask=(tokenized_sentence['attention_mask']).to(device)

  output = model.generate(
      input_ids=input_ids,
      attention_mask=attention_mask,
      max_length=512,
      num_beams=5,
      early_stopping=True,
  )

  # Decode the output tokens to text
  decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)
  return decoded_output

In [ ]:
def pre(sentence):
  return sentence.lower().strip()
def post(sentence):
  return sentence.upper()

predicted_outputs = []
index = 0
for sentence_batch in X_eval_splitted:
  index += 1
  curr_batch = []
  for sentence in sentence_batch:
    output_text = infer(model,pre(sentence))
    print("   ",output_text)
    cleaned_text = output_text.replace("<|endoftext|>", "")
    cleaned_text = post(cleaned_text)
    curr_batch.append(cleaned_text)
  predicted_output = ' '.join(curr_batch)
  predicted_outputs.append(predicted_output)
  print(index, " / ", len(X_eval_splitted))
  # print(predicted_output)

predicted_outputs

    बड़े बच्चों के नाभी का रणना क्या होता है
1  /  1001
    गाय के अफारा लोग से कैसे बचाओगे
2  /  1001
    कर्थ नक्त की नुकसान की प्रवरिती और निशान बताईं
3  /  1001
    गेहुत की और सिंचिदसा प्रजाति के बारे में बताएं
4  /  1001
    जौफ के जूजा या पत्ती बूढ़ी हैं
5  /  1001
    पिला रश्ट हर दा का जैविक नाश्ते कैसे करेंगे
6  /  1001
    मुंगकी पंत मुंग एक किसने की जानकारी दी
7  /  1001
    जठिंडा की सड़क दुर्घटना किसने के बारे में बताईं
8  /  1001
    लूख उसमट रोग से नियंत्रन की रफ्फ गडगडकरी
9  /  1001
    हर हर दाल का भी पालन कैसे करनी चाहिए
10  /  1001
    जौर के पेड़ों को बिरली कर नीले रंग के लोग दीजिए
11  /  1001
    गडब की आसा प्रजातितित दसा प्रजाति के बारे में बिडियो
12  /  1001
    टॉमटर के पवदों पर मिट्टी चढ़ना क्यूस
13  /  1001
    टॉम रोकने की रोक रोकथाम कैसे करनी चाहिए
14  /  1001
    सिवेरियम रोग के पर चिंतित लोगों के नाम बताएं
15  /  1001
    अमरोद की पौधा रोपन विधी के बारे में बिकिनी
16  /  1001
    रणबीर की खेती के लिए खेत रोपन विधी बताईं
17  /  1001
    सरिफा की बूतों की द

['बड़े बच्चों के नाभी का रणना क्या होता है',
 'गाय के अफारा लोग से कैसे बचाओगे',
 'कर्थ नक्त की नुकसान की प्रवरिती और निशान बताईं',
 'गेहुत की और सिंचिदसा प्रजाति के बारे में बताएं',
 'जौफ के जूजा या पत्ती बूढ़ी हैं',
 'पिला रश्ट हर दा का जैविक नाश्ते कैसे करेंगे',
 'मुंगकी पंत मुंग एक किसने की जानकारी दी',
 'जठिंडा की सड़क दुर्घटना किसने के बारे में बताईं',
 'लूख उसमट रोग से नियंत्रन की रफ्फ गडगडकरी',
 'हर हर दाल का भी पालन कैसे करनी चाहिए',
 'जौर के पेड़ों को बिरली कर नीले रंग के लोग दीजिए',
 'गडब की आसा प्रजातितित दसा प्रजाति के बारे में बिडियो',
 'टॉमटर के पवदों पर मिट्टी चढ़ना क्यूस',
 'टॉम रोकने की रोक रोकथाम कैसे करनी चाहिए',
 'सिवेरियम रोग के पर चिंतित लोगों के नाम बताएं',
 'अमरोद की पौधा रोपन विधी के बारे में बिकिनी',
 'रणबीर की खेती के लिए खेत रोपन विधी बताईं',
 'सरिफा की बूतों की देख रेग किसी करने चाहिए',
 'जो मैं नाइट्रोजन हूँ फासफोल्त रणबीर कपूर और रणबीर कपूर',
 'मुर्गी खेतों के लिए बुमराह की सड़क दुर्घटना बताईं',
 'मुससुर दाल के बीच छेड़छाड़ की बारे में कहानियाँ',
 'सुखा 

In [ ]:
# join prediected_outputs with '\n'
print(predicted_outputs)
english_out = '\n'.join(predicted_outputs)
english_out

['बड़े बच्चों के नाभी का रणना क्या होता है', 'गाय के अफारा लोग से कैसे बचाओगे', 'कर्थ नक्त की नुकसान की प्रवरिती और निशान बताईं', 'गेहुत की और सिंचिदसा प्रजाति के बारे में बताएं', 'जौफ के जूजा या पत्ती बूढ़ी हैं', 'पिला रश्ट हर दा का जैविक नाश्ते कैसे करेंगे', 'मुंगकी पंत मुंग एक किसने की जानकारी दी', 'जठिंडा की सड़क दुर्घटना किसने के बारे में बताईं', 'लूख उसमट रोग से नियंत्रन की रफ्फ गडगडकरी', 'हर हर दाल का भी पालन कैसे करनी चाहिए', 'जौर के पेड़ों को बिरली कर नीले रंग के लोग दीजिए', 'गडब की आसा प्रजातितित दसा प्रजाति के बारे में बिडियो', 'टॉमटर के पवदों पर मिट्टी चढ़ना क्यूस', 'टॉम रोकने की रोक रोकथाम कैसे करनी चाहिए', 'सिवेरियम रोग के पर चिंतित लोगों के नाम बताएं', 'अमरोद की पौधा रोपन विधी के बारे में बिकिनी', 'रणबीर की खेती के लिए खेत रोपन विधी बताईं', 'सरिफा की बूतों की देख रेग किसी करने चाहिए', 'जो मैं नाइट्रोजन हूँ फासफोल्त रणबीर कपूर और रणबीर कपूर', 'मुर्गी खेतों के लिए बुमराह की सड़क दुर्घटना बताईं', 'मुससुर दाल के बीच छेड़छाड़ की बारे में कहानियाँ', 'सुखा जड़ सरन की इस भाग में

'बड़े बच्चों के नाभी का रणना क्या होता है\nगाय के अफारा लोग से कैसे बचाओगे\nकर्थ नक्त की नुकसान की प्रवरिती और निशान बताईं\nगेहुत की और सिंचिदसा प्रजाति के बारे में बताएं\nजौफ के जूजा या पत्ती बूढ़ी हैं\nपिला रश्ट हर दा का जैविक नाश्ते कैसे करेंगे\nमुंगकी पंत मुंग एक किसने की जानकारी दी\nजठिंडा की सड़क दुर्घटना किसने के बारे में बताईं\nलूख उसमट रोग से नियंत्रन की रफ्फ गडगडकरी\nहर हर दाल का भी पालन कैसे करनी चाहिए\nजौर के पेड़ों को बिरली कर नीले रंग के लोग दीजिए\nगडब की आसा प्रजातितित दसा प्रजाति के बारे में बिडियो\nटॉमटर के पवदों पर मिट्टी चढ़ना क्यूस\nटॉम रोकने की रोक रोकथाम कैसे करनी चाहिए\nसिवेरियम रोग के पर चिंतित लोगों के नाम बताएं\nअमरोद की पौधा रोपन विधी के बारे में बिकिनी\nरणबीर की खेती के लिए खेत रोपन विधी बताईं\nसरिफा की बूतों की देख रेग किसी करने चाहिए\nजो मैं नाइट्रोजन हूँ फासफोल्त रणबीर कपूर और रणबीर कपूर\nमुर्गी खेतों के लिए बुमराह की सड़क दुर्घटना बताईं\nमुससुर दाल के बीच छेड़छाड़ की बारे में कहानियाँ\nसुखा जड़ सरन की इस भाग में लगता है\nसुखा जडरोग से नियंत्रन की जैविक प

In [ ]:
OUT_FILE_PATH = '/content/drive/MyDrive/ASR_EVAL_DATA/hindi_mbart.txt'
f = open(OUT_FILE_PATH, "w")
f.write(english_out)
f.close()